# Notebook para filtrar o arquivo conllu em sentenças que contenham pelo menos duas URIS anotadas 

In [5]:
import argparse
import json
from conllu import parse
import pandas as pd
from conllu import parse_incr
import pickle
import numpy as np
import re
import os
import unicodedata
import numpy as np

### Funções 

In [6]:
#utils
def get_start(x, offset):
    start = str(int(x["start_char"]) - offset)
    return start
def get_end(x, offset):
    end = str(int(x["end_char"]) - offset)
    return end
def correct_start_end(df):
    offset = int(df.iloc[0]["misc"].get("start_char"))
    df['start'] = df['misc'].apply(lambda x: get_start(x, offset))
    df['end'] = df['misc'].apply(lambda x: get_end(x, offset))
    return df

def remove_null_tokens(df):#retira tokens nulos na sentenca
    df = df[df['misc'].notna()]
    df = df.reset_index(drop=True)
    df = df[df['deps'].notna()]
    df = df.reset_index(drop=True)
    return df

def ents_uris_counter_prefiltering(sentence,ents,uris): #contagem de entidades e uris pre-filtragem
    entidades_sentenca_old, uris_sentenca_old = ents, uris
    ents_list, uris_list = ents, uris
    for idx_row in range(len(sentence)):
        token = sentence.iloc[idx_row]['deps']
        grafo = sentence.iloc[idx_row]['misc'].get('grafo')
        if "B=" in token:
            ents_list.append(token)
        if grafo:
            uris_list.append(grafo)     
    return ents_list, uris_list

def verify_sentence_is_functioning(df_sentence):
    if df_sentence.iloc[0]["misc"].get('eliminar') == '':#retirar linha com eliminar
        print('eliminada')
        return 0
    for idx_row in range(len(df_sentence)):
        if df_sentence.iloc[idx_row]['misc'].get('start_char') == None:#problema com start_char
            print('start_char com problema')
            return 0
    return 1

def get_text_sentence(df):
    size_sentence = int(df_sentence.iloc[-1]["end"])
    text = " "*size_sentence
    for index, row in df_sentence.iterrows():
        text = text[:int(row["start"])] + row["form"] +text[int(row["end"]):]
    return text 

def get_word_join(df, index):
    entity = df.iloc[index]['deps']
    entity_I = entity.replace("B=","I=")
    count = 1
    word_join = ""
    row_main = df.iloc[index]
    start_word, end_word = row_main['start'], row_main['end'] 
    word_join = " ".join([word_join, row_main['form']])
    while index+count != len(df) and (df.iloc[index+count]["deps"] == entity_I or check_I_entities(df, index+count,entity_I)):
        row = df.iloc[index+count]
        word_join = " ".join([word_join, row['form']])
        end_word = row['end']
        count+=1
    return word_join, start_word, end_word

def check_I_entities(df, i,entity):
    #verifica se ainda está dentro da entidade usando o sistema BIO de tokens
    next_entity_is_I = (df.iloc[i]["deps"] == entity) or (df.iloc[i]["deps"] == None and df.iloc[i+1]["deps"] == entity)
    return next_entity_is_I

def check_sentence_for_pair_uris(df_sentence, ents, uris):
    df_sent_new = pd.DataFrame()
    ents_list_old, uris_list_old = ents, uris
    ents_list, uris_list = ents, uris
    
    countURIs = 0
    for idx_row in range(len(df_sentence)):
        token = df_sentence.iloc[idx_row]['deps']
        grafo = df_sentence.iloc[idx_row]['misc'].get('grafo')
        if 'B=' in token and grafo:
            countURIs+=1
            ents_list = ents_list + [token]
            uris_list = uris_list + [grafo]
            
            df_sentence.loc[idx_row,'grafo'] = grafo
            df_sentence.loc[idx_row,'text'] = get_text_sentence(df_sentence)
            word_join, start_word_join, end_word_join = get_word_join(df_sentence, idx_row)
            df_sentence.loc[idx_row,'word_join'] = word_join
            df_sentence.loc[idx_row,'word_join_start'] = start_word_join
            df_sentence.loc[idx_row,'word_join_end'] = end_word_join
            df_sentence.loc[idx_row,'index_e'] = int(idx_row)
            
            df_sent_new = df_sent_new.append(df_sentence.iloc[idx_row],ignore_index = True)
#             raise SystemExit("Stop right there!")   
    if countURIs > 1:
        return 1, df_sent_new, ents_list, uris_list
    return 0, df_sentence, ents_list_old, uris_list_old,

## Ler conllu para filtrar sentencas que contenham pelo menos duas URIs

In [8]:
#CONLLU_PATH = "../PetroNER/petroner-uri-2023-04-05.conllu"
CONLLU_PATH = "../PetroNER/petroner-uri-validação.conllu"
data_file = open(CONLLU_PATH, "r", encoding="utf-8")
sentences=[]
for tokenlist in parse_incr(data_file):
    sentences.append(tokenlist)
print('Total number of sentences in conllu ->',len(sentences))
#algum problema nessas sentencas em diferentes petroner
sentences_with_issues = [12408, 13636, 15264, 21023, 21122, 23920, 24017]
# sentences_with_issues = [1749, 4747, 4935, 5066, 5235, 5798, 6802, 7858, 8881, 9271, 10689, 10691, 10695, 10750, 
#                          11144, 12408, 13393, 13636, 15264, 17494, 18462, 18476, 19346, 21023, 21105, 21122, 
#                         22163, 23920, 24017]

Total number of sentences in conllu -> 2467


## Rotina para contar entidades e URIs no conllu (é necessário rodar)

In [9]:
df_new = pd.DataFrame()
contSentences = 0
ents, uris = [], []
for idxSentence in range(0,len(sentences)):
    #if idxSentence not in sentences_with_issues:
    try:
        #print('sentence = ' ,idxSentence)
        sentence = sentences[idxSentence]
        json_sent = json.dumps(sentence)
        df_sentence = pd.read_json(json_sent)
        df_sentence = remove_null_tokens(df_sentence) 
        if verify_sentence_is_functioning(df_sentence):
            df_sentence = correct_start_end(df_sentence)    
            
            checkQtdTokenURI, df_test, ents, uris = check_sentence_for_pair_uris(df_sentence,ents,uris)
            if checkQtdTokenURI == 1:
                contSentences+= 1
                df_test['sentence'] = contSentences
                df_test['#sentence_original'] = idxSentence
                df_new = df_new.append(df_test)
                       
                #print('sentences after filtering ->', contSentences)
#                 raise SystemExit("Stop right there!")
    except:
        pass

df_new.drop(columns=["start","end","head","id","feats","upos","lemma","xpos"], inplace=True)

#pickle.dump(df_new, open('df_filtred_petroner_uri_2023_04_05.conllu.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
#df_new.to_csv('df_filtred_petroner_uri_2023_04_05_conllu.csv',encoding = 'utf-8',index=False)

pickle.dump(df_new, open('df_filtred_petroner_uri_valid.conllu.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
df_new.to_csv('df_filtred_petroner_uri_valid_conllu.csv',encoding = 'utf-8',index=False)


#entidades é a lista de diferentes tipos de entidades que aparecem após filtragem das sentencas
entidades, numb_ents = np.unique(ents, return_counts = True)
#grafos é a lista de diferentes URIs anotadas que aparecem após filtragem das sentencas
grafos, numb_grafos = np.unique(uris, return_counts = True)
print('------------')
print('lista de diferentes tipos de entidades pós filtragem')
print(entidades.tolist())
print('------------')
print('quantidades de diferentes tipos de entidades pós filtragem')
print(numb_ents.tolist())
print('------------')
print('Total de URIs pós filtragem ->', sum(numb_grafos.tolist()))

start_char com problema
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
------------
lista de diferentes tipos de entidades pós filtragem
['B=BACIA', 'B=CAMPO', 'B=ESTRUTURA_FÍSICA', 'B=FLUIDO', 'B=FLUIDODATERRA_i', 'B=FLUIDODATERRA_o', 'B=NÃOCONSOLID', 'B=POÇO', 'B=POÇO_Q', 'B=POÇO_T', 'B=ROCHA', 'B=TIPO_POROSIDADE', 'B=UNIDADE_CRONO', 'B=UNIDADE_LITO']
------------
quantidades de diferentes tipos de entidades pós filtragem
[278, 41, 100, 8, 38, 8, 102, 98, 1, 1, 216, 12, 279, 82]
------------
Total de URIs pós filtragem -> 1264


## Rotina para contar entidades e URIS no arquivo conllu (não é necessário rodar)

In [5]:
ents_pre = []
uris_pre = []
for idxGroups in range(0,len(sentences)):
    if idxGroups not in sentences_with_issues: 
        #print('sentence = ' ,idxGroups)
        sentence = sentences[idxGroups]
        json_temp = json.dumps(sentence)
        df_get_start_end = pd.read_json(json_temp)
        df_get_start_end = remove_null_tokens(df_get_start_end) 
        if verify_sentence_is_functioning(df_get_start_end):
            df_get_start_end = correct_start_end(df_get_start_end) 
        
            ents_pre, uris_pre = ents_uris_counter_prefiltering(df_get_start_end,ents_pre,uris_pre)
        
entidades_pre, numb_ents_pre = np.unique(ents_pre, return_counts = True)
grafos_pre, numb_grafos_pre = np.unique(uris_pre, return_counts = True)
print('------------')
print('lista de diferentes tipos de entidades no conllu')
print(entidades_pre.tolist())
print('------------')
print('quantidades de diferentes tipos de entidades no conllu')
print(numb_ents_pre.tolist())
print('------------')
print('Total de URIs no Conllu original ->',sum(numb_grafos_pre.tolist()))

eliminada
eliminada
eliminada
start_char com problema
eliminada
start_char com problema
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
eliminada
el